# Linear Regression

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns

The idea of _correlation_ is the simple idea that variables often change _together_. For a simple example, cities with more buses tend to have higher populations.

We might observe that, as one variable X increases, so does another Y, OR that as X increases, Y decreases.

The _covariance_ describes how two variables co-vary. Note the similarity in the definition to the definition of ordinary variance:

## Covariance

For two random variables $X$ and $Y$, each with $n$ values:

$\Large\sigma_{XY} = \frac{\Sigma^n_{i = 1}(x_i - \mu_x)(y_i - \mu_y)}{n}$ <br/>

In [2]:
X = [1, 3, 5]
Y = [2, 9, 10]

# Covariance by hand:
((1-3) * (2-7) + (3-3) * (9-7) + (5-3) * (10-7)) / 3

5.333333333333333

In [3]:
# Better yet: With NumPy:
np.cov(X, Y, ddof=0)[0, 1]

5.333333333333333

In [4]:
np.cov(X, Y, ddof=0)

array([[ 2.66666667,  5.33333333],
       [ 5.33333333, 12.66666667]])

In [5]:
np.var(Y)

12.666666666666666

Note that the value of the covariance is very much a function of the values of X and Y, which can make interpretation difficult. What is wanted is a _standardized_ scale for covariance, hence: _correlation_.

## Correlation

Pearson Correlation:<br/>$\Large r_P = \frac{\Sigma^n_{i = 1}(x_i - \mu_x)(y_i - \mu_y)}{\sqrt{\Sigma^n_{i = 1}(x_i - \mu_x)^2\Sigma^n_{i = 1}(y_i -\mu_y)^2}}$

Note that we are simply standardizing the covariance by the standard deviations of X and Y (the $n$'s cancel!).

$\bf{Check}$:

<details><summary>
What happens if X = Y?
</summary>
Then numerator = denominator and the correlation = 1!
</details>
<br/>
We'll always have $-1 \leq r \leq 1$. (This was the point of standardizing by the standard deviations of X and Y.)

A correlation of -1 means that X and Y are perfectly negatively correlated, and a correlation of 1 means that X and Y are perfectly positively correlated.

NumPy also has a correlation method:

In [6]:
np.corrcoef(X, Y)

array([[1.        , 0.91766294],
       [0.91766294, 1.        ]])

In [7]:
4 / np.sqrt(19)

0.917662935482247

In [8]:
np.corrcoef(X, Y)[0, 1] == (np.cov(X, Y, ddof=0) / (np.std(X) * np.std(Y)))[0, 1]

True

And so does SciPy:

In [9]:
stats.pearsonr(X, Y)

(0.917662935482247, 0.2601469382930062)

## Causation

_Why_ does it happen that variables correlate? It _may_ be that one is the cause of the other. A city having a high population, for example, probably does have some causal effect on the number of buses that the city has. But this _need not_ be the case, and that is why statisticians are fond of saying that 'correlation is not causation'. An alternative possibility, for example, is that high values of X and Y are _both_ caused by high values of some third factor Z. The size of children's feet, for example, is correlated with their ability to spell, but this is of course NOT because either is a cause of the other. Rather, BOTH are caused by the natural maturing and development of children. As they get older, both their feet and their spelling abilities grow!

## Statistical Learning Theory

It's important at this point to understand the distinction between dependent and independent variables.

Roughly, the independent variable is what can be directly manipulated and the dependent variable is what cannot be (but is nevertheless of great interest). What matters structurally is simply that we understand the dependent variable to be a _function_ of the independent variable(s).

This is the proper interpretation of a statistical _model_.

Simple idea: We can model correlation with a _line_. As one variable changes, so does the other.

This model has two *parameters*: *slope* and *y-intercept*.

Unless there's a perfectly (and suspiciously) linear relationship between our predictor(s) and our target, there will  be some sort of **error** or **loss** or **residual**. The best-fit line is constructed by minimizing the sum of the squares of these losses.

## Simple Linear Regression

The solution for a simple regression best-fit line is as follows:

- slope: <br/>$\Large m = r_P\frac{\sigma_y}{\sigma_x}$

- y-intercept:<br/> $\Large b = \mu_y - m\mu_x$

## Assumptions of Linear Regression:

### 1. The relationship between target and predictor(s) is linear. (Of course!)

**How can I check for this?**
- Build a scatterplot of y vs. various predictors.

**What can I do if it looks like I'm violating this assumption?**
- Consider log-scaling your data.
- Consider a different type of model!

### 2. The errors are mutually independent. (That is, there is no correlation between any two errors.)

**How can I check for this?**
- Build an error plot, i.e. a plot of errors for a particular predictor (vs. the values of that predictor).

**What can I do if it looks like I'm violating this assumption?**
- Consider dropping extreme values.

### 3. The errors are normally distributed. (That is, smaller errors are more probable than larger errors, according to the familiar bell curve.)

**How can I check for this?**
- Check the Omnibus value (More on this later!)
- Check the Jarque-Bera value (More on this later!)
- Build a QQ-Plot.

**What can I do if it looks like I'm violating this assumption?**
- Consider log-scaling your data.

### 4. The errors are homoskedastic. (That is, the errors have the same variance.)

**How can I check for this?**
- Check the Durbin-Watson score (More on this later!)
- Conduct a Goldfeld-Quandt test.
- Build an error plot, i.e. a plot of errors for a particular predictor (vs. the values of that predictor).

**What can I do if it looks like I'm violating this assumption?**
- Consider dropping extreme values.
- Consider log-scaling your target.
- Consider a different type of model!

There is no general requirement that the predictors and the target *themselves* be normally distributed. However: Linear regression can work better if the predictors and target are normally distributed. Log-scaling can be a good tool to make data more normal.

Suppose e.g. a kde plot of one of my predictors $X_1$ looks like this:

![original](images/skewplot.png)

In that case the kde plot of $log(X_1)$ looks like this:

![log](images/logplot.png)

[Here](https://www.statisticssolutions.com/assumptions-of-linear-regression/) is a helpful resource on the assumptions of linear regression.

Experiment: [Playing with regression line](https://www.desmos.com/calculator/jwquvmikhr) <br/>
Limitations: [Anscombe's Quartet](https://www.desmos.com/calculator/paknt6oneh)

# Using Sklearn

**No Intercept:**

In [10]:
from sklearn.linear_model import LinearRegression
df = pd.DataFrame([X,Y]).T
df.columns = ['X', 'Y']
model = LinearRegression(fit_intercept=False)
model.fit(df[['X']], df.Y)

LinearRegression(fit_intercept=False)

**Returning the coefficient:**

In [11]:
model.coef_

array([2.25714286])

**With Intercept**

In [12]:
model = LinearRegression()
model.fit(df[['X']], df.Y)

LinearRegression()

In [13]:
print('Coefficient:', model.coef_)
print('Intercept:', model.intercept_)

Coefficient: [2.]
Intercept: 1.0000000000000018


# Using OLS

### Using Python objects 

No intercept:

In [14]:
import statsmodels.api as sm

model = sm.OLS(Y,X)
results = model.fit()

results.params

array([2.25714286])

With Intercept:

In [15]:
from statsmodels.tools import add_constant
x_plus_intercept = add_constant(X)
model = sm.OLS(Y, x_plus_intercept)
results = model.fit()
results.params

array([1., 2.])

### Using the formula option

No intercept:

In [16]:
df = pd.DataFrame([X,Y]).T
df.columns = ['X', 'Y']
formula = 'Y~X-1'
model = sm.OLS.from_formula(formula, data=df)
results = model.fit()
results.params

X    2.257143
dtype: float64

With Intercept:

In [17]:
formula = 'Y~X+1'
model = sm.OLS.from_formula(formula, data=df)
results = model.fit()
results.params

Intercept    1.0
X            2.0
dtype: float64

**Results Tables:**

In [18]:
results.summary()

C:\Users\angel\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:70: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.684
Method:                 Least Squares   F-statistic:                     5.333
Date:                Thu, 25 Feb 2021   Prob (F-statistic):              0.260
Time:                        08:28:46   Log-Likelihood:                -5.2965
No. Observations:                   3   AIC:                             14.59
Df Residuals:                       1   BIC:                             12.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0000      2.958      0.338      0.792     -36.585      38.585
X              2.0000      0.866      2.309      0.260      -9.004      13.004
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.000
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                           0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         7.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Visualize simple Linear Regression

In [19]:
def best_line(X, Y):
    """This function plots the best-fit line for a group of datapoints broken into
    a set of independent variable values (X) and a set of dependent variable values (Y)"""
    
    # Create a linear regression model using 
    # either Sklearn or Statsmodels

    # Collect beta_1 and beta_0
    beta_1 = None
    beta_0 = None
    
    # Plot the best fit line!
    Xs = np.linspace(np.min(X), np.max(X), 100)
    Ys = beta_1 * Xs + beta_0
    plt.scatter(X, Y, label='datapoints', c='red')
    plt.plot(Xs, Ys, 'k', label=f'y={round(beta_1, 2)}x+{round(beta_0, 2)}')
    plt.legend(loc='upper left', framealpha=1)
    plt.show();

In [20]:
# Using best_line
best_line(X, Y)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
ans = sns.load_dataset('anscombe')
sns.scatterplot(data=ans, x='x', y='y');

In [ ]:
best_line(ans.x, ans.y)

The best-fit line exists no matter what my data look like!

In [ ]:
X_rand = stats.uniform.rvs(size=100)
Y_rand = stats.uniform.rvs(size=100)
plt.scatter(X_rand, Y_rand, c='red');

In [ ]:
best_line(X_rand, Y_rand)

# Homework
>Code Simple Linear Regression from Scratch

<img src="images/equation.png" style="width:400px">

In [ ]:
def best_line(X, Y):
    """This function plots the best-fit line for a group of datapoints broken into
    a set of independent variable values (X) and a set of dependent variable values (Y)"""
   
    # Calculate the means for both arrays

    
    # Calculate the difference between each observation
    # and the means for both arrays

    # Multiple the differences together

    
    # Sum the squared differences for each array
    # and multiple them together.
    # Take the square root of their product.

    
    # Calculate the correlation coefficient

    
    # Multiple the correlation coefficient
    # by the standard deviation of y 
    # divided by the standard deviation of x

    
    # Multiply beta_1 and the mean of the X
    # Subtract the product from the mean of Y

    
    # Plot the best fit line!
    Xs = np.linspace(np.min(X), np.max(X), 100)
    Ys = beta_1 * Xs + beta_0
    plt.scatter(X, Y, label='datapoints', c='red')
    plt.plot(Xs, Ys, 'k', label=f'y={round(beta_1, 2)}x+{round(beta_0, 2)}')
    plt.legend(loc='upper left', framealpha=1)
    plt.show();